This Jupyter notebook compares four neural network methods (GHNN, SympNet, HenonNet, and Reservoir Computing) for modeling the Duffing oscillator, a chaotic Hamiltonian system. It generates Figures 3 and 4 
 **“Phase Space Integrity in Neural Network Models of Hamiltonian Dynamics: A Lagrangian Descriptor Approach.”**
by:

1. Loading pre-trained models and 160,000 test trajectories
2. Generating bidirectional predictions (forward and backward in time) from initial conditions near homoclinic orbits
3. Computing prediction errors in position and momentum
4. Calculating Lagrangian Descriptors (LD) to quantify trajectory divergence and estimate Lyapunov exponents
5. Creating multi-panel plots showing:
Position/momentum errors vs. time
Error-to-LD ratios
LD/time (Lyapunov exponent proxy)

The figures illustrates the process of short term error prediction and the corresponding LDs in representative points. 

In [ ]:
# Import required packages for reservoir computing, plotting, and data handling
using HDF5            # For reading/writing HDF5 data files
using SparseArrays    # For efficient sparse matrix operations
using Pkg
Pkg.activate("D:\\Documents\\KU\\Dissertation\\Project4\\ReservoirComputing")

using Plots           # Plotting library
using LaTeXStrings    # LaTeX formatting in plots
using ReservoirComputing  # Echo State Network (ESN) implementation


  Activating project at `D:\Documents\KU\Dissertation\Project4\ReservoirComputing`


In [ ]:
# Configure Python environment for calling GHNN/SympNet/HenonNet models
ENV["JULIA_CONDAPKG_BACKEND"] = "Null"
# Set Python executable path (update with your actual username path if needed)
ENV["JULIA_PYTHONCALL_EXE"] = "C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\python.exe"
ENV["PATH"] = "C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312;C:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python312\\DLLs;" * ENV["PATH"]
# Add GHNN module to Python path
ENV["PYTHONPATH"] = raw"D:\Documents\KU\Dissertation\Project4\generalized_hnn\GHNN"
using PythonCall



"D:\\Documents\\KU\\Dissertation\\Project4\\generalized_hnn\\GHNN"

In [ ]:
# Load RC/ESN model parameters and weights from an HDF5 group.
# Supports both sparse triplet format (I, J, V) and dense matrix storage.
# Returns: esn (Echo State Network), W_out (forward output weights), W_out_rev (backward output weights)
function read_model(model,test_data; leaky = 1.0)
    res_size = read_attribute(model,"res_size")
    # Load input matrix (either sparse or dense format)
    if haskey(model["input_matrix"],"I")
        
        I1 = read(model["input_matrix"],"I")
        J1 = read(model["input_matrix"],"J")
        X1 = read(model["input_matrix"],"V")
        input_matrix = sparse(I1,J1,X1)
    else
        input_matrix = read(model["input_matrix"],keys(model["input_matrix"])[1])
    end
    
    # Load reservoir matrix (either sparse or dense format)
    if haskey(model["reservoir_matrix"],"I")
        I2 = read(model["reservoir_matrix"],"I")
        J2 = read(model["reservoir_matrix"],"J")
        X2 = read(model["reservoir_matrix"],"V")
        reservoir_matrix = sparse(I2,J2,X2,res_size,res_size)
    else
        reservoir_matrix= read(model["reservoir_matrix"],keys(model["reservoir_matrix"])[1])
    end
        
    # Load output weights for forward and backward predictions    
    W_out= read(model,"Wout")
	W_out_rev= read(model,"Wout_rev")

    # Create ESN with leaky coefficient α and washout period
    esn = ESN(test_data, reservoir_matrix ,input_matrix;
            reservoir_driver = RNN(leaky_coefficient = α),washout = wash)
    return esn,W_out,W_out_rev
end

# Combine forward and backward neural network predictions into a single trajectory.
# Reverses backward predictions and concatenates them with forward predictions.
# Args: forward/backward - DataFrames from GHNN/SympNet/HenonNet predictions
#       dict_states - mapping of state indices to column names (e.g., "q_A", "p_A")
function combine_forward_backward_predict(forward, backward, dict_states)
    Nel = length(dict_states)
    # Extract forward prediction values
    U_GH = [pyconvert(Vector,forward[dict_states[i]].values ) for i in 1:Nel]
    U_GH = hcat(U_GH...)'
    # Extract and reverse backward prediction values (excluding last point to avoid duplication)
    U_GHb = [reverse(pyconvert(Vector,backward[dict_states[i]].values ))[1:end-1] for i in 1:Nel]
    U_GHb = hcat(U_GHb...)'
    # Concatenate: [backward_reversed | forward]
    UU_GHNN_combined = hcat(U_GHb,U_GH)
    return UU_GHNN_combined
end

# Combine forward and backward RC predictions with initial condition in the middle.
# Structure: [backward_reversed | x0 | forward]
function combine_forward_backward_predict_RC(forward, backward, x0)
    RC_combined = hcat(reverse(backward, dims = 2),x0,forward)
    return RC_combined
end

# Distance metric for Lagrangian Descriptor (LD) calculation.
# Uses only the first two state components (position and momentum).
# Args: u1, u2 - state vectors to compare
#       p - exponent for the norm (default=1)
function distance_vector(u1,u2,p = 1)
    d_a = sum(abs.(u1[1:2]-u2[1:2]).^p)
    return d_a
end


distance_vector (generic function with 2 methods)

In [ ]:
# Import Python modules for neural network models
ghnn = pyimport("ghnn");
np = pyimport("numpy");
pd = pyimport("pandas")

# Load pre-trained neural network models for Duffing oscillator
# GHNN: Generalized Hamiltonian Neural Network (energy-conserving)
nn_paths = "duffing_200/GHNN/nn_1_infer_eq/";
ghnn_model= ghnn.nets.net_from_dir(nn_paths);

# SympNet: Symplectic Neural Network (preserves symplectic structure)
sympnet_path = "duffing_200/SympNet/nn_1/";
sympnet_model =ghnn.nets.net_from_dir(sympnet_path);

# HenonNet: Volume-preserving neural network
henon_path = "duffing_200/HenonNet/nn_1/";
henon_model = ghnn.nets.net_from_dir(henon_path);

# Map state indices to variable names (q=position, p=momentum for species A)
dict_state = Dict(1 => "q_A", 2 => "p_A")
dict_ylabel = Dict(1 => "q", 2 => "p")


Dict{Int64, String} with 2 entries:
  2 => "p"
  1 => "q"

In [ ]:
# Load HDF5 data files containing test trajectories
fid = h5open("ESN_data_test2.h5", "r");  # RC model parameters
train_test_file = h5open("rnn_test_ld_duff_w200.h5","r")  # Forward trajectories
train_test_file_back = h5open("rnn_test_ld_duff_w200_back.h5","r")  # Backward trajectories
test1 = read(train_test_file,"test_1")  # Load first test case


2×201 Matrix{Float64}:
 -1.3764   -1.26996  -1.15569  …  -1.19683  -1.30881  -1.41141   -1.5
  1.01055   1.11062   1.1683      -1.15229  -1.08032  -0.963923  -0.8

In [ ]:
# Configure RC model parameters
wash = 200;  # Washout period (transient steps to discard)
RC_new_combined = h5open("ESN_data_test2.h5", "r");
RCmodel = keys(RC_new_combined)[14]  # Select 14th RC model from file
QP_esn = RC_new_combined[RCmodel]

# Extract leaky coefficient (α) from model metadata
alp = read_attribute(QP_esn,"res_driver")
i1 = findlast(",",alp)
α = parse(Float64,alp[i1[1]+1:end-1])

# Load RC model with extracted parameters
esn_cycle_c,W_out_cycle_c,W_out_cycle_c_back = read_model(QP_esn,test1,leaky = α);

# Time integration parameters
tend = 10   # End time for trajectory
dt = 0.1    # Time step
Nt = Int(tend/dt+1)  # Number of time points



101

In [ ]:
# Load all initial conditions from the 160,000 test cases
# Each test case has an initial condition (x0,y0) stored as an attribute
init_cond = Array{Float64}(undef,160000,2)
for i in 1:160000
    init_cond[i,:] = read_attribute(train_test_file_back["test_$i"],"(x0,y0)")
end



In [ ]:
# Configure plot labels and method styles for comparison
dict_ylabel = Dict(1 => "q- \\hat{q}", 2 => "p- \\hat{p}", 3 => "e", 4 => L"LD/\tau")
# Each method: (name, linestyle, marker, color_index, linewidth)
methods = [
    ("data", :solid,:none,1,5.5),
    ("SympNet", :dot  ,:diamond,2,2.5),
    ("HenonNet",:dashdotdot,:utriangle,3,2.5),
    ("GHNN",    :dashdot,:square,4,2.5),
    ("RC",   :dash,:circle,5,2.5)
]
t_jump = 5  # Plotting interval (plot every 5th point)
t = -20:0.1:20  # Time array from -20 to 20
dt = 0.1
gamma = 0.7  # Exponent for LD calculation

# Main plotting function: compares prediction errors and LD across methods
# Args: u0s - array of initial conditions to plot
# Returns: array of plot objects for multi-panel figure
function plot_results_error(u0s)
    
    nplot = size(u0s, 1)  # Number of initial conditions to plot
    plot_array = Array{Any}(undef,4*nplot + 1)  # 4 columns per row + legend
    
    for j in 1:nplot
        # Find the test case closest to the requested initial condition
        _, d = findmin(sum(abs.(init_cond' .- u0s[j]), dims=1))
        i = d[2]

        # Load backward and forward test data
        test1 = read(train_test_file_back, "test_$i")
        test1f = read(train_test_file, "test_$i")
        data = hcat(test1f,reverse(test1[:,1:end-1], dims = 2));  # Combined trajectory
        
        # Generate RC predictions (backward and forward)
        predict_states = create_states(esn_cycle_c.reservoir_driver, test1, wash,
                                    esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                    esn_cycle_c.bias_vector)
        UU = esn_cycle_c(Generative(Nt-1), W_out_cycle_c_back, predict_states[:,end])

        predict_statesf = create_states(esn_cycle_c.reservoir_driver, test1f, wash,
                                        esn_cycle_c.reservoir_matrix, esn_cycle_c.input_matrix,
                                        esn_cycle_c.bias_vector)
        UUf = esn_cycle_c(Generative(Nt-1), W_out_cycle_c, predict_statesf[:,end])

        # Generate neural network predictions (forward and backward)
        x0 = np.array(test1f[:,end])
        UU_GHNN_combined  = combine_forward_backward_predict(ghnn_model.predict_path(x0,tend),
                                                            ghnn_model.predict_path(x0,-tend), dict_state)
        UU_SympNet_combined = combine_forward_backward_predict(sympnet_model.predict_path(x0,tend),
                                                                sympnet_model.predict_path(x0,-tend), dict_state)
        UU_Henon_combined = combine_forward_backward_predict(henon_model.predict_path(x0,tend),
                                                            henon_model.predict_path(x0,-tend), dict_state)
        UU_RC_combined = combine_forward_backward_predict_RC(UUf, UU, pyconvert(Vector, x0))
        
        # Store all method predictions
        struct_data = [data[:,101:301], UU_SympNet_combined, UU_Henon_combined, UU_GHNN_combined, UU_RC_combined]

        # Calculate errors relative to data
        err = Array{Vector{Float64}}(undef,4)  # Distance error
        err_p = Array{Vector{Float64}}(undef,4)  # Momentum error
        err_q = Array{Vector{Float64}}(undef,4)  # Position error
        err_glob =  Array{Vector{Float64}}(undef,5)  # LD for all methods
        
        for i in 1:5
            if i!=1
                # Error between prediction and reference data
                err[i-1] = [distance_vector(struct_data[i][:,j],struct_data[1][:,j],0.7) for j in 1:size(struct_data[1],2)]
                err_p[i-1] = [struct_data[i][1,j]-struct_data[1][1,j] for j in 1:size(struct_data[1],2)]
                err_q[i-1] = [struct_data[i][2,j]-struct_data[1][2,j] for j in 1:size(struct_data[1],2)]
            end
            # LD: distance between consecutive time steps
            err_glob[i] = [distance_vector(struct_data[i][:,j],struct_data[i][:,j+1],0.7) for j in 1:size(struct_data[i],2)-1]
        end

        # Cumulative LD calculations
        ld_d1 = Vector{Vector{Float64}}(undef,4)
        ld_glob = Vector{Vector{Float64}}(undef,5)
        ld_glob[1] = cumsum(err_glob[1][101:200]+reverse(err_glob[1][1:100]))*dt^(1-gamma)
        for i in 2:5
            ld_d1[i-1] = cumsum(err[i-1][102:201]+reverse(err[i-1][1:100]))*dt
            ld_glob[i] = cumsum(err_glob[i][101:200]+reverse(err_glob[i][1:100]))*dt^(1-gamma)
        end
        
        # === Create 4-column plot for this initial condition ===
        for ax1 in 1:4  # Loop over columns

            p = plot()
            if ax1 ==1  # Column 1: Position error
                for i in 2:5
                    if i ==1 
                        p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
                    else
                        p = plot!(t[101:t_jump:301], err_p[i-1][1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2],
                                lw =  methods[i][5], ms =3)
                    end
                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(-10,10), ylim=(-1,1), tick_direction=:out)
                
            elseif ax1 == 2  # Column 2: Momentum error
                for i in 2:5
                    if i ==1 
                        p = plot!(t[101:301], data[ax1,101:301], color=1, lw=4, label="data")
                    else
                        p = plot!(t[101:t_jump:301], err_q[i-1][1:t_jump:end],label=methods[i][1],
                                color=methods[i][4], linestyle=methods[i][2], 
                                lw =  methods[i][5], ms =3)
                    end
                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(-10,10), ylim=(-1,1), tick_direction=:out)
                
            elseif ax1 == 3  # Column 3: Error/LD ratio
                for i in 2:5
                    p = plot!(t[202:t_jump:301], ld_d1[i-1][1:t_jump:end]./ld_glob[1][1:t_jump:end],label=methods[i][1],
                            color=methods[i][4], linestyle=methods[i][2],
                            lw =  methods[i][5], ms =3)
                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(0,10), ylim=(0,1), tick_direction=:out)
                
            else  # Column 4: LD/τ (Lyapunov exponent estimate)
                for i in 1:5
                    p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] ,label=methods[i][1],
                            color=methods[i][4], linestyle=methods[i][2],
                            lw =  methods[i][5], ms = 3)
                end
                plot!(ylabel=latexstring("$(dict_ylabel[ax1])"), legend=:false,
                xlim=(0,10), ylim=(1,3), tick_direction=:out)
            end
            
            p = plot!(legend = false)  # Hide legend in individual plots
            
            # Add x-axis label to bottom row
            if j == nplot
                if ax1 < 3
                    xlabel!(L"t")
                else
                    xlabel!(L"\tau")
                end
            end
            plot_array[(j-1)*4+ax1] = p
        end

        # Add legend panel at the end
        if j == nplot
            p = plot()
            for i in 1:5
                p = plot!(t[202:t_jump:301], ld_glob[i][1:t_jump:end]./t[202:t_jump:301] ,label=methods[i][1],
                        color=methods[i][4], linestyle=methods[i][2],
                        lw =  methods[i][5], ms = 3)
            end
            plot!(xlim=(0,10), ylim=(1,3), tick_direction=:out)
            p = plot!(legend = :outerbottom, legendcolumns=5, axis = false)
            plot_array[end] = p
        end
    end
    return plot_array
end
    



plot_results_error (generic function with 1 method)

In [ ]:
# Plot comparison for 3 initial conditions near homoclinic orbit (negative momentum)
u0s = [[-0.71,-0.51],[-0.71,-0.61],  [-0.71,-0.71]]
pl_err_array = plot_results_error(u0s)
l = @layout [grid(3,4); a{0.085h}]  # 3 rows × 4 columns + legend row
plot(pl_err_array..., layout = l,size = (1200,500),dpi = 300, left_margin = 4Plots.mm)
savefig("time_trace_homoclinic_Duffing_err.svg")


GKS: Rectangle definition is invalid in routine SET_VIEWPORT
GKS: Rectangle definition is invalid in routine SET_VIEWPORT


"d:\\Documents\\KU\\Dissertation\\Project4\\Duffing\\time_trace_homoclinic_Duffing_err.svg"

In [ ]:
# Plot comparison for 3 initial conditions near homoclinic orbit (positive momentum)
u0s = [[-0.71,0.51],[-0.71,0.61],  [-0.71,0.71]]
pl_err_array = plot_results_error(u0s)
l = @layout [grid(3,4); a{0.085h}]  # 3 rows × 4 columns + legend row
plot(pl_err_array..., layout = l,size = (1200,500),dpi = 300, left_margin = 4Plots.mm)
savefig("time_trace_homoclinic_Duffing_err_plus.svg")


GKS: Rectangle definition is invalid in routine SET_VIEWPORT
GKS: Rectangle definition is invalid in routine SET_VIEWPORT


"d:\\Documents\\KU\\Dissertation\\Project4\\Duffing\\time_trace_homoclinic_Duffing_err_plus.svg"